In [ ]:
print("\n" + "="*60)
print("Benchmark Summary")
print("="*60)

print(f"\nTest Configuration:")
print(f"  Number of test cases: {len(test_cases)}")
print(f"  Spot price: ${test_cases[0]['spot']:.2f}")
print(f"  Strike range: ${min(tc['strike'] for tc in test_cases):.0f} - ${max(tc['strike'] for tc in test_cases):.0f}")
print(f"  Grid points: {base_params['num_space_points']}")
print(f"  Volatility: {base_params['volatility']:.4f}")

print(f"\nBlack-Scholes Prices:")
print(f"  Min price: ${bs_df['bs_price'].min():.6f}")
print(f"  Max price: ${bs_df['bs_price'].max():.6f}")
print(f"  Mean price: ${bs_df['bs_price'].mean():.6f}")

print(f"\nExpected Convergence:")
print(f"  From {grid_sizes[0]} to {grid_sizes[-1]} points:")
print(f"  Error reduction: {convergence_errors[0]['error']/convergence_errors[-1]['error']:.1f}x")
print(f"  Expected rate: O(h²) (second order)")

print("\nNext Steps:")
print("  1. Implement the fixed-grid PDE solver")
print("  2. Implement the adaptive time-stepping solver")
print("  3. Re-run benchmarks to compare accuracy vs computational time")
print("="*60)

## 5. Summary Statistics

Compute and display key benchmark metrics.

In [ ]:
# Plot 1: Price profiles across different strikes
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Black-Scholes prices across strikes
ax = axes[0, 0]
strikes_range = np.linspace(80, 120, 50)
bs_prices = [call_price(100, s, base_params['rate'], base_params['volatility'], 
                        base_params['time_to_expiry']) for s in strikes_range]
ax.plot(strikes_range, bs_prices, 'b-', linewidth=2, label='Black-Scholes')
ax.axvline(100, color='k', linestyle='--', alpha=0.3, label='ATM')
ax.set_xlabel('Strike Price')
ax.set_ylabel('Option Price ($)')
ax.set_title('Black-Scholes Call Price Profile')
ax.grid(True, alpha=0.3)
ax.legend()

# Convergence plot
ax = axes[0, 1]
ax.loglog(conv_df['grid_size'], conv_df['error'], 'o-', linewidth=2, markersize=8, label='Actual')
ax.loglog(conv_df['grid_size'], 10/(conv_df['grid_size'])**2, '--', alpha=0.5, label='O(h²)')
ax.set_xlabel('Grid Size (number of points)')
ax.set_ylabel('Absolute Error')
ax.set_title('Expected Convergence Rate')
ax.grid(True, alpha=0.3, which='both')
ax.legend()

# Benchmark comparison table summary
ax = axes[1, 0]
ax.axis('off')
table_data = bs_df[['strike', 'moneyness', 'bs_price']].copy()
table_data.columns = ['Strike', 'Moneyness', 'BS Price']
table = ax.table(cellText=table_data.values, colLabels=table_data.columns,
                cellLoc='center', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)
ax.set_title('Black-Scholes Reference Prices', pad=20)

# Time to expiry sensitivity
ax = axes[1, 1]
times = np.linspace(0.1, 2.0, 30)
call_by_time = [call_price(100, 100, base_params['rate'], base_params['volatility'], t) 
                for t in times]
ax.plot(times, call_by_time, 'g-', linewidth=2)
ax.set_xlabel('Time to Expiry (years)')
ax.set_ylabel('ATM Call Price ($)')
ax.set_title('ATM Call Price vs Time to Expiry')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Visualizations

Create comparison plots between solvers.

In [ ]:
# Test convergence with different grid sizes
print("\n" + "="*60)
print("Convergence Analysis")
print("="*60)

grid_sizes = [51, 101, 151, 201, 301]
convergence_errors = []

# Use ATM case for convergence analysis
atm_strike = 100.0
atm_price = call_price(
    spot=100.0,
    strike=atm_strike,
    rate=base_params['rate'],
    volatility=base_params['volatility'],
    time_to_expiry=base_params['time_to_expiry']
)

print(f"\nATM Reference Price: ${atm_price:.6f}")
print("\nNote: PDE solvers not yet fully implemented - showing expected convergence pattern\n")

# Simulate convergence (for when solvers are implemented)
for n in grid_sizes:
    # Simulated error (would be actual error when solvers implemented)
    simulated_error = 10.0 / (n * n)  # O(h²) convergence
    convergence_errors.append({
        'grid_size': n,
        'error': simulated_error
    })

conv_df = pd.DataFrame(convergence_errors)
print(conv_df.to_string(index=False))

## 3. Error Analysis

Analyze convergence and error behavior.

In [ ]:
# Calculate Black-Scholes benchmark prices
print("\nCalculating Black-Scholes benchmark prices...")
bs_results = []

for tc in test_cases:
    bs_price = call_price(
        spot=tc['spot'],
        strike=tc['strike'],
        rate=base_params['rate'],
        volatility=base_params['volatility'],
        time_to_expiry=base_params['time_to_expiry']
    )
    bs_results.append({
        'strike': tc['strike'],
        'moneyness': 'OTM' if tc['spot'] < tc['strike'] else ('ATM' if tc['spot'] == tc['strike'] else 'ITM'),
        'bs_price': bs_price
    })

bs_df = pd.DataFrame(bs_results)
print(bs_df.to_string(index=False))

# Store for later comparison
landmark_prices = {row['strike']: row['bs_price'] for _, row in bs_df.iterrows()}

## 2. Run Benchmark Comparison

Execute solvers and compare results.

In [ ]:
# Define benchmark parameters
print("="*60)
print("Benchmark Configuration")
print("="*60)

# Base parameters for all solvers
base_params = {
    'rate': 0.05,
    'volatility': 0.2,
    'time_to_expiry': 1.0,
    'num_space_points': 201
}

print(f"Risk-free Rate:  {base_params['rate']:.4f}")
print(f"Volatility:      {base_params['volatility']:.4f}")
print(f"Time to Expiry:  {base_params['time_to_expiry']:.4f} years")
print(f"Grid Points:     {base_params['num_space_points']}")

# Create diverse test cases
test_cases = []
strikes = [80, 90, 100, 110, 120]
for strike in strikes:
    test_cases.append({
        'spot': 100.0,
        'strike': strike,
        'option_type': 'call'
    })

print(f"\nTest Cases: {len(test_cases)}")
for i, tc in enumerate(test_cases, 1):
    moneyness = "OTM" if tc['spot'] < tc['strike'] else ("ATM" if tc['spot'] == tc['strike'] else "ITM")
    print(f"  {i}. Spot={tc['spot']:.0f}, Strike={tc['strike']:.0f} ({moneyness})")
print("="*60)

## 1. Load and Run Benchmarks

Define benchmark parameters and run comparison solvers.

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# Add src directory to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from bs_analytic import call_price, put_price
from benchmarks import compare_solvers
from config import PDEConfig

# Benchmark Results: PDE Solver Comparison

This notebook compares the fixed-grid and adaptive PDE solvers against the analytical Black-Scholes solution.